# Network Analysis for BeveL Betaseries 

Inputs: betaseries files for BeveL participants (n=85) drawn from 4 conditions: choice, reward taste, punishment taste, neutral rinse

Analysis workflow is mapped off this paper: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5429248/

In [1]:
import glob
import os
import networkx as nx
import numpy as np
import pandas as pd
import bz2
import pickle
import community
import statistics
import pdb
from scipy import stats
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt

#from visbrain.objects import ConnectObj, SceneObj, SourceObj, BrainObj
#from visbrain.io import download_file

## Load in the data

### Find the path to the data

In [2]:
#Find the path to data
file_list = glob.glob('/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/reward/*.txt')

In [3]:
#Check the files found
print(file_list)

['/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/reward/sub-059_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/reward/sub-010_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/reward/sub-027_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/reward/sub-072_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/reward/sub-045_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/reward/sub-054_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/reward/sub-063_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/reward/sub-036_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/reward/sub-001_reward.txt', '/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/reward/sub-048_reward.txt', '/Users/jennygilbert/Document

In [4]:
#Check to see how many participants 
len(file_list)

85

### Make a dictionary to read in the files to pandas

In [5]:
my_dict={}
for item in file_list:
    name=item.split('/')[7].split('.')[0]
    print(name)
    my_dict.setdefault(name, []).append(item)

sub-059_reward
sub-010_reward
sub-027_reward
sub-072_reward
sub-045_reward
sub-054_reward
sub-063_reward
sub-036_reward
sub-001_reward
sub-048_reward
sub-080_reward
sub-066_reward
sub-004_reward
sub-033_reward
sub-085_reward
sub-018_reward
sub-009_reward
sub-022_reward
sub-015_reward
sub-040_reward
sub-077_reward
sub-088_reward
sub-025_reward
sub-012_reward
sub-047_reward
sub-070_reward
sub-039_reward
sub-028_reward
sub-082_reward
sub-061_reward
sub-056_reward
sub-003_reward
sub-034_reward
sub-087_reward
sub-078_reward
sub-053_reward
sub-064_reward
sub-031_reward
sub-006_reward
sub-017_reward
sub-020_reward
sub-075_reward
sub-042_reward
sub-069_reward
sub-067_reward
sub-050_reward
sub-005_reward
sub-032_reward
sub-019_reward
sub-084_reward
sub-089_reward
sub-014_reward
sub-041_reward
sub-076_reward
sub-058_reward
sub-090_reward
sub-011_reward
sub-026_reward
sub-073_reward
sub-044_reward
sub-055_reward
sub-062_reward
sub-037_reward
sub-081_reward
sub-079_reward
sub-086_reward
sub-052_re

In [6]:
# Checking to make sure its populated
my_dict['sub-024_reward']

['/Users/jennygilbert/Documents/betaseries_bevel/4_combine_timeseries/reward/sub-024_reward.txt']

### Read the data from the dictionary into pandas

In [7]:
data_dict={}
for key, value in my_dict.items():
    for i in value:
        data_dict.setdefault(key, []).append(pd.read_csv(i, sep='\t' ,header=None,index_col=False))
#         data_dict[key]= pd.read_csv(i, header=None,index_col=False)

In [8]:
#Check the dictionary
data_dict.keys()

dict_keys(['sub-059_reward', 'sub-010_reward', 'sub-027_reward', 'sub-072_reward', 'sub-045_reward', 'sub-054_reward', 'sub-063_reward', 'sub-036_reward', 'sub-001_reward', 'sub-048_reward', 'sub-080_reward', 'sub-066_reward', 'sub-004_reward', 'sub-033_reward', 'sub-085_reward', 'sub-018_reward', 'sub-009_reward', 'sub-022_reward', 'sub-015_reward', 'sub-040_reward', 'sub-077_reward', 'sub-088_reward', 'sub-025_reward', 'sub-012_reward', 'sub-047_reward', 'sub-070_reward', 'sub-039_reward', 'sub-028_reward', 'sub-082_reward', 'sub-061_reward', 'sub-056_reward', 'sub-003_reward', 'sub-034_reward', 'sub-087_reward', 'sub-078_reward', 'sub-053_reward', 'sub-064_reward', 'sub-031_reward', 'sub-006_reward', 'sub-017_reward', 'sub-020_reward', 'sub-075_reward', 'sub-042_reward', 'sub-069_reward', 'sub-067_reward', 'sub-050_reward', 'sub-005_reward', 'sub-032_reward', 'sub-019_reward', 'sub-084_reward', 'sub-089_reward', 'sub-014_reward', 'sub-041_reward', 'sub-076_reward', 'sub-058_reward',

In [9]:
#Check for the dataframe
data_dict['sub-058_reward']

[             0           1           2           3           4          5   \
 0    115.271950   48.284798  -24.895542   40.839394   35.163708  -9.680651   
 1     -6.440373   66.152054  -62.975662   13.151779    1.603159   8.119939   
 2    -97.147217  279.945587  -36.686951  -53.525486   39.616070 -19.276733   
 3    178.807526  186.360535   36.817814   50.739586   18.883062  40.366669   
 4     68.360291   45.647396  -16.965845   84.948692   39.198357 -30.621855   
 5   -224.359589 -178.982056    3.218035  -28.598566  -64.436050 -36.720451   
 6    102.598129   17.282980  -26.879036  -86.759956   39.177406  13.087736   
 7     76.307739   29.793316   10.971916   58.552471   -0.441348  -0.808222   
 8    210.930359  159.814880   45.043007  133.201599  104.217514 -29.165623   
 9    -91.944801   84.479836    4.762266    9.267733   37.287460  23.768517   
 10    15.600883    7.233562  -85.792770   49.226746    8.287991 -54.691212   
 11  -251.825546 -104.581917   49.624496   -5.594570

### Create a new dictionary with correlation matrix

In [10]:
cor_dict={}

for key, value in data_dict.items():
    value[0]
    #pdb.set_trace()
    cor_matrix = value[0].corr()
    cor_dict[key] = cor_matrix
    

In [11]:
#check the dictionary
list(cor_dict.values())[3]

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,1.000000,0.251398,-0.162775,-0.294740,-0.186054,-0.408077,0.542839,0.729705,0.286417,0.401842,...,-0.084741,0.146889,0.293645,0.530057,-0.150380,-0.042867,-0.651930,-0.129725,0.087826,0.243320
1,0.251398,1.000000,-0.601794,-0.350516,-0.070828,0.102098,0.161121,0.347082,0.153182,-0.083195,...,-0.377490,0.135588,-0.110900,0.183797,-0.177225,0.118628,0.044018,0.091710,0.012343,0.125064
2,-0.162775,-0.601794,1.000000,0.492950,0.317473,0.029196,-0.144105,-0.024520,0.357031,0.421046,...,0.684577,-0.273778,0.550258,0.282005,0.372391,0.345070,0.006756,0.038023,0.617744,0.352090
3,-0.294740,-0.350516,0.492950,1.000000,-0.230818,0.281308,0.122457,-0.086780,0.162486,-0.033059,...,0.482912,-0.283151,-0.233378,-0.242601,0.093543,0.312408,-0.118387,0.162622,0.345256,-0.071805
4,-0.186054,-0.070828,0.317473,-0.230818,1.000000,0.013006,-0.243825,-0.355588,-0.029757,-0.085314,...,0.220390,-0.208114,0.459847,0.319668,0.536979,0.367359,0.142260,-0.154733,0.305493,0.223647
5,-0.408077,0.102098,0.029196,0.281308,0.013006,1.000000,0.124754,-0.069164,0.282740,-0.233923,...,-0.141000,0.156732,-0.244294,-0.437554,-0.034353,0.056889,0.325948,-0.021583,-0.098977,-0.304306
6,0.542839,0.161121,-0.144105,0.122457,-0.243825,0.124754,1.000000,0.662701,0.397289,0.165634,...,-0.147428,0.125791,-0.070707,0.026247,-0.335394,-0.081640,-0.481591,0.300650,0.047996,-0.172870
7,0.729705,0.347082,-0.024520,-0.086780,-0.355588,-0.069164,0.662701,1.000000,0.599560,0.478848,...,0.056400,0.308574,0.376366,0.475062,-0.213484,0.031686,-0.377789,0.240086,0.346111,0.313296
8,0.286417,0.153182,0.357031,0.162486,-0.029757,0.282740,0.397289,0.599560,1.000000,0.581443,...,0.277958,-0.188422,0.557129,0.419469,0.016059,0.448080,-0.316262,0.175506,0.546295,0.426270
9,0.401842,-0.083195,0.421046,-0.033059,-0.085314,-0.233923,0.165634,0.478848,0.581443,1.000000,...,0.236948,0.077210,0.611002,0.458380,-0.137031,0.186530,-0.298651,0.326962,0.494723,0.601267


### Make a dictionary of labels for the nodes

In [12]:
path = '/Users/jennygilbert/Documents/betaseries_bevel/5_analysis/labels.txt'
df_label = pd.read_csv(path, sep=',')

df_label.head()

#labels = [line.split(',') for line in open('/Users/jennygilbert/Documents/betaseries_bevel/5_analysis/labels.txt', 'r').readlines()]
#print(labels)

,Amygdala_L,Amygdala_R,Dorsal_striatum_L,Dorsal_striatum_R,Fusiform_gyrus_L,Fusiform_gyrus_R,Hippocampus_L,Hippocampus_R,Insula_L,Insula_R,...,Precuneus_L,Precuneus_R,Ventral_striatum_L,Ventral_striatum_R,vlPFC_L,vlPFC_R,vlThalamus_L,vlThalamus_R,vmPFC_L,vmPFC_R


In [13]:
labels_dict = {}
n=0
for item in df_label:
    labels_dict[n]=item
    n=n+1

In [14]:
print(labels_dict)

{0: 'Amygdala_L', 1: 'Amygdala_R', 2: 'Dorsal_striatum_L', 3: 'Dorsal_striatum_R', 4: 'Fusiform_gyrus_L', 5: 'Fusiform_gyrus_R', 6: 'Hippocampus_L', 7: 'Hippocampus_R', 8: 'Insula_L', 9: 'Insula_R', 10: 'Intracalcarine_cortex_L', 11: 'Intracalcarine_cortex_R', 12: 'lOFC_L', 13: 'lOFC_R', 14: 'mOFC_L', 15: 'mOFC_R', 16: 'Oral_somatosensory_cortex_L', 17: 'Oral_somatosensory_cortex_', 18: 'Precuneus_L', 19: 'Precuneus_R', 20: 'Ventral_striatum_L', 21: 'Ventral_striatum_R', 22: 'vlPFC_L', 23: 'vlPFC_R', 24: 'vlThalamus_L', 25: 'vlThalamus_R', 26: 'vmPFC_L', 27: 'vmPFC_R'}


### Function to create a graph with positive or negative values and minimum correlation value

In [15]:
def create_corr_network_5(G, corr_direction, min_correlation):

    ##Creates a copy of the graph
    H = G.copy()
    
    ##Checks all the edges and removes some based on corr_direction
    for stock1, stock2, weight in list(G.edges(data=True)):
        ##if we only want to see the positive correlations we then delete the edges with weight smaller than 0        
        if corr_direction == "positive":
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] <0 or weight["weight"] < min_correlation:
                H.remove_edge(stock1, stock2)
        ##this part runs if the corr_direction is negative and removes edges with weights equal or largen than 0
        else:
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] >=0 or weight["weight"] > min_correlation:
                H.remove_edge(stock1, stock2)
    return(H)

### Function to make a graph object BY SUBJECT
This will return:
- The edges (noramlized R correlation matrix, in pandas dataframe)
- The correlations (absolute value of the edges in a numpy dataframe)
- The mean_FC (the mean functional connectivity per subject/run)
- The graphs (this will contain the raw graph object G as well as the the partion values from the modularity calculation)

In [16]:
def make_graphs(list_o_data, direction, min_cor):
    edge_dict={}
    cor_dict={}
    FC_dict={}
    graph_dict={}
    partition_dict={}
    for key, values in list_o_data.items():
            #i=i.set_index(labels.ID)
            #i.rename(columns=labels.ID, inplace=True)
            ########################################
            edge_dict.setdefault(key, []).append(values)
            ########################################
            cor_matrix = np.asmatrix(values)
            x=abs(cor_matrix)
            mu=x.mean()
            ########################################
            cor_dict.setdefault(key, []).append(x)
            ########################################
            FC_dict.setdefault(key, []).append(mu)
            ########################################
            G = nx.from_numpy_matrix(cor_matrix)
            #for i, nlrow in labels.iterrows():
                #G.node[i].update(nlrow[0:].to_dict())
            ########################################
            graph_dict.setdefault(key, []).append(G)
            ########################################
            partition = community.best_partition(create_corr_network_5(G, direction,min_cor))
            ########################################
            partition_dict.setdefault(key, []).append(partition)
            ########################################
    return({'edges':edge_dict, 'correlations':cor_dict, 'mean_FC':FC_dict, 'graphs':graph_dict, 'modules':partition_dict})

### Apply the function to correlations & check output

In [17]:
# Apply function
GRAPHS = make_graphs(cor_dict, "positive", 0)

In [18]:
# Check the keys for the dictionary
GRAPHS.keys()

dict_keys(['edges', 'correlations', 'mean_FC', 'graphs', 'modules'])

In [19]:
# Check modules for one subject
GRAPHS['modules']['sub-001_reward']

[{0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 4,
  5: 4,
  6: 0,
  7: 0,
  8: 1,
  9: 1,
  10: 4,
  11: 4,
  12: 4,
  13: 1,
  14: 4,
  15: 4,
  16: 2,
  17: 2,
  18: 1,
  19: 1,
  20: 5,
  21: 5,
  22: 1,
  23: 2,
  24: 1,
  25: 1,
  26: 5,
  27: 2}]

In [20]:
#Check to make sure graphs are filled
Test = GRAPHS['graphs']['sub-001_reward'][0]
Test.edges(data=True)

EdgeDataView([(0, 0, {'weight': 1.0}), (0, 1, {'weight': -0.3972298556919801}), (0, 2, {'weight': 0.27312137454636276}), (0, 3, {'weight': 0.18893460566107687}), (0, 4, {'weight': 0.0737438000730521}), (0, 5, {'weight': 0.05161801239223985}), (0, 6, {'weight': 0.35007533783335354}), (0, 7, {'weight': 0.31038996583346107}), (0, 8, {'weight': -0.2632553566380778}), (0, 9, {'weight': -0.15855482756962685}), (0, 10, {'weight': 0.03354937287845262}), (0, 11, {'weight': 0.06843617993524459}), (0, 12, {'weight': 0.15234399003643884}), (0, 13, {'weight': -0.04960639850984408}), (0, 14, {'weight': 0.01989172938970772}), (0, 15, {'weight': 0.07153526676144391}), (0, 16, {'weight': 0.07988892360209846}), (0, 17, {'weight': 0.24518793187244337}), (0, 18, {'weight': -0.1841693897955745}), (0, 19, {'weight': 0.10202674772067095}), (0, 20, {'weight': 0.23868106477936482}), (0, 21, {'weight': 0.01152342340090486}), (0, 22, {'weight': -0.06953283500392686}), (0, 23, {'weight': 0.026017359160952575}), (

In [21]:
GRAPHS['mean_FC']

{'sub-059_reward': [0.28417982480041054],
 'sub-010_reward': [0.286494156389793],
 'sub-027_reward': [0.4087092701281954],
 'sub-072_reward': [0.29876319400123935],
 'sub-045_reward': [0.4037509712670373],
 'sub-054_reward': [0.30217725903440273],
 'sub-063_reward': [0.30872213370358403],
 'sub-036_reward': [0.2853068206610137],
 'sub-001_reward': [0.2848602794212194],
 'sub-048_reward': [0.22649027734012997],
 'sub-080_reward': [0.3007339386554587],
 'sub-066_reward': [0.3365612245251495],
 'sub-004_reward': [0.21269017088271103],
 'sub-033_reward': [0.27804698772816977],
 'sub-085_reward': [0.34250173173223997],
 'sub-018_reward': [0.26954354391005303],
 'sub-009_reward': [0.2965987229797016],
 'sub-022_reward': [0.2552437366975302],
 'sub-015_reward': [0.29886493927261676],
 'sub-040_reward': [0.2637785299819304],
 'sub-077_reward': [0.2818497864623579],
 'sub-088_reward': [0.3580383502336406],
 'sub-025_reward': [0.2682607227565468],
 'sub-012_reward': [0.3585074814383908],
 'sub-0

### Get the standard deviation of the mean FC

In [22]:
#statistics.stdev(GRAPHS['mean_FC'])
o=[]

for key,value in GRAPHS['mean_FC'].items():
    o.append(value[0])

In [23]:
statistics.stdev(o)

0.06429119953348585

## ^ What do we do with this??? 
If this value is low (<0.1?) then you don't need to threshold a graph

### Test Modularity

In [24]:
#Goal: use modularity function from communities to identify module structure that 
#emerge during reward within the sample

In [25]:
#this is pulling out the module structure for each participant
modules=[]

for key,value in GRAPHS['modules'].items():
    modules.append(value[0])

In [26]:
#print it to make sure it works
modules

[{0: 0,
  1: 1,
  2: 2,
  3: 2,
  4: 3,
  5: 4,
  6: 0,
  7: 0,
  8: 5,
  9: 5,
  10: 4,
  11: 4,
  12: 3,
  13: 4,
  14: 3,
  15: 5,
  16: 5,
  17: 5,
  18: 4,
  19: 4,
  20: 6,
  21: 6,
  22: 1,
  23: 1,
  24: 5,
  25: 5,
  26: 6,
  27: 1},
 {0: 0,
  1: 1,
  2: 2,
  3: 2,
  4: 3,
  5: 0,
  6: 3,
  7: 4,
  8: 1,
  9: 1,
  10: 0,
  11: 0,
  12: 5,
  13: 1,
  14: 4,
  15: 4,
  16: 5,
  17: 5,
  18: 6,
  19: 1,
  20: 7,
  21: 7,
  22: 5,
  23: 5,
  24: 1,
  25: 1,
  26: 4,
  27: 7},
 {0: 0,
  1: 1,
  2: 0,
  3: 2,
  4: 1,
  5: 3,
  6: 4,
  7: 2,
  8: 5,
  9: 2,
  10: 3,
  11: 3,
  12: 1,
  13: 2,
  14: 3,
  15: 3,
  16: 0,
  17: 0,
  18: 5,
  19: 5,
  20: 5,
  21: 4,
  22: 5,
  23: 5,
  24: 2,
  25: 2,
  26: 5,
  27: 5},
 {0: 0,
  1: 1,
  2: 2,
  3: 2,
  4: 3,
  5: 4,
  6: 0,
  7: 0,
  8: 0,
  9: 5,
  10: 0,
  11: 3,
  12: 2,
  13: 4,
  14: 0,
  15: 0,
  16: 2,
  17: 2,
  18: 2,
  19: 4,
  20: 5,
  21: 5,
  22: 3,
  23: 3,
  24: 4,
  25: 4,
  26: 2,
  27: 5},
 {0: 0,
  1: 0,
  2: 1,
  3:

In [27]:
#now summarize the module belonging for each node

In [28]:
df = pd.DataFrame.from_dict(modules)

#Rows = participants
#Colums = nodes

In [29]:
df

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,0,1,2,2,3,4,0,0,5,5,...,4,4,6,6,1,1,5,5,6,1
1,0,1,2,2,3,0,3,4,1,1,...,6,1,7,7,5,5,1,1,4,7
2,0,1,0,2,1,3,4,2,5,2,...,5,5,5,4,5,5,2,2,5,5
3,0,1,2,2,3,4,0,0,0,5,...,2,4,5,5,3,3,4,4,2,5
4,0,0,1,2,3,4,2,5,1,1,...,2,2,1,0,1,1,3,3,1,1
5,0,0,1,2,3,4,0,0,5,5,...,6,6,3,3,1,1,2,2,3,5
6,0,0,1,1,2,3,0,0,4,1,...,6,6,0,0,5,6,0,4,7,7
7,0,0,1,1,2,1,0,0,3,3,...,3,3,2,3,1,0,4,4,2,2
8,0,1,2,3,4,4,0,0,1,1,...,1,1,5,5,1,2,1,1,5,2
9,0,0,1,1,2,3,0,0,2,2,...,2,2,5,0,5,5,2,6,5,1


In [30]:
df.rename(columns={0:"Amygdala_L",1:"Amygdala_R", 2:"Dorsal_striatum_L", 3:"Dorsal_striatum_R", 4:"Fusiform_gyrus_L", 5:"Fusiform_gyrus_R", 6:"Hippocampus_L", 7:"Hippocampus_R", 8:"Insula_L",
          9:"Insula_R", 10:"Intracalcarine_cortex_L", 11:"Intracalcarine_cortex_R", 12:"lOFC_L", 13: "lOFC_R", 14: "mOFC_L", 15:"mOFC_R", 16:"Oral_somatosensory_cortex_L", 17:"Oral_somatosensory_cortex_R", 18:"Precuneus_L", 
          19:"Precuneus_R", 20:"Ventral_striatum_L", 21:"Ventral_striatum_R", 22:"vlPFC_L", 23:"vlPFC_R", 24:"vlThalamus_L" , 25:"vlThalamus_R", 26: "vmPFC_L", 27: "vmPFC_R"})

,Amygdala_L,Amygdala_R,Dorsal_striatum_L,Dorsal_striatum_R,Fusiform_gyrus_L,Fusiform_gyrus_R,Hippocampus_L,Hippocampus_R,Insula_L,Insula_R,...,Precuneus_L,Precuneus_R,Ventral_striatum_L,Ventral_striatum_R,vlPFC_L,vlPFC_R,vlThalamus_L,vlThalamus_R,vmPFC_L,vmPFC_R
0,0,1,2,2,3,4,0,0,5,5,...,4,4,6,6,1,1,5,5,6,1
1,0,1,2,2,3,0,3,4,1,1,...,6,1,7,7,5,5,1,1,4,7
2,0,1,0,2,1,3,4,2,5,2,...,5,5,5,4,5,5,2,2,5,5
3,0,1,2,2,3,4,0,0,0,5,...,2,4,5,5,3,3,4,4,2,5
4,0,0,1,2,3,4,2,5,1,1,...,2,2,1,0,1,1,3,3,1,1
5,0,0,1,2,3,4,0,0,5,5,...,6,6,3,3,1,1,2,2,3,5
6,0,0,1,1,2,3,0,0,4,1,...,6,6,0,0,5,6,0,4,7,7
7,0,0,1,1,2,1,0,0,3,3,...,3,3,2,3,1,0,4,4,2,2
8,0,1,2,3,4,4,0,0,1,1,...,1,1,5,5,1,2,1,1,5,2
9,0,0,1,1,2,3,0,0,2,2,...,2,2,5,0,5,5,2,6,5,1


In [31]:
#find the mean over the columns
df.mean(axis = 0)

0     0.000000
1     0.352941
2     1.211765
3     1.352941
4     1.611765
5     2.164706
6     1.164706
7     1.082353
8     2.341176
9     2.317647
10    2.517647
11    2.600000
12    2.635294
13    2.670588
14    2.235294
15    2.400000
16    1.847059
17    1.811765
18    2.094118
19    2.352941
20    2.400000
21    2.305882
22    2.435294
23    2.576471
24    2.623529
25    2.682353
26    2.447059
27    2.564706
dtype: float64

In [32]:
# Check the standard deviation over the columns
df.std(axis = 0)

0     0.000000
1     0.480721
2     0.558359
3     0.718893
4     1.186171
5     1.316987
6     1.446235
7     1.440996
8     1.484467
9     1.415995
10    1.323881
11    1.311125
12    1.454444
13    1.475097
14    1.485221
15    1.513432
16    1.349655
17    1.286321
18    1.823285
19    1.723133
20    1.827045
21    1.915000
22    1.721344
23    1.741486
24    1.799315
25    1.794249
26    1.714823
27    1.679336
dtype: float64

In [33]:
#find the mean over the columns
partition_median = df.median()
df_median = pd.DataFrame(partition_median)

In [34]:
# View the median module for each node
df_median

,0
0,0.0
1,0.0
2,1.0
3,1.0
4,2.0
5,2.0
6,0.0
7,0.0
8,2.0
9,2.0


### Intermediate Results
In response to reward, nodes can be partitioned into four modules: 

0 = 0:"Amygdala_L", 1:"Amygdala_R", 6:"Hippocampus_L", 7:"Hippocampus_R"  

1 = 2:"Dorsal_striatum_L", 3:"Dorsal_striatum_R", 16:"Oral_somatosensory_cortex_L",17:"Oral_somatosensory_cortex_R"

2 = 4:"Fusiform_gyrus_L", 5:"Fusiform_gyrus_R", 8:"Insula_L", 9:"Insula_R", 14: "mOFC_L", 15:"mOFC_R", 18:"Precuneus_L", 19:"Precuneus_R", 20:"Ventral_striatum_L", 21:"Ventral_striatum_R", 22:"vlPFC_L", 23:"vlPFC_R", 24:"vlThalamus_L" , 25:"vlThalamus_R", 26: "vmPFC_L", 27: "vmPFC_R"

3 = 10:"Intracalcarine_cortex_L", 11:"Intracalcarine_cortex_R", 12:"lOFC_L", 13: "lOFC_R"

### Label the nodes in the graph with ROI names

In [35]:
for graph in GRAPHS['graphs'].values(): 
    for i in graph: 
        nx.set_node_attributes(i, labels_dict, 'ROIs')

### Visualize Results

In [36]:
def create_corr_network(G, *argv):
    # creates a list for edges and for the weights
    # G must be a networkx graph object
    # Style = Choice of graph = circle, spring, spectral
    
    edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
    
    
    #positions
    if len(argv) != 0:
        for arg in argv:
            print("Making %s graph"%arg)
            if arg == "circle":
                positions=nx.circular_layout(G)
            elif arg == "spring":
                positions=nx.spring_layout(G)
            elif arg == "spectral":
            
                positions=nx.spectral_layout(G)
    else:
        positions=nx.circular_layout(G)

    #Figure size
    plt.figure(figsize=(15,15))

    #draws nodes
    nx.draw_networkx_nodes(G,positions,node_color='#DA70D6',
                           node_size=500,alpha=0.8)
    labels=dict(G.nodes(data=True))
    #Styling for labels
    
    
    nx.draw_networkx_labels(G,positions,labels, font_size=8, 
                            font_family='sans-serif')
        
    #draws the edges
    vmin = min(weights)
    vmax = max(weights)
    
    w=np.array(weights)*10
    nx.draw_networkx_edges(G, positions, edge_list=edges, style='solid', width= w, edge_color= weights, 
                           edge_cmap=plt.cm.seismic, vmin=vmin, vmax=vmax)
    
    # displays the graph without axis
    plt.axis('off')
    #saves image
    #plt.savefig("part1.png", format="PNG")
    sm = plt.cm.ScalarMappable(cmap=plt.cm.seismic, norm=plt.Normalize(vmin = vmin, vmax=vmax))
    sm._A = []
    plt.colorbar(sm)
    plt.show() 

In [37]:
print(GRAPHS['graphs'].values())

dict_values([[<networkx.classes.graph.Graph object at 0x1a17edd080>], [<networkx.classes.graph.Graph object at 0x1a17edd588>], [<networkx.classes.graph.Graph object at 0x1a17edd4a8>], [<networkx.classes.graph.Graph object at 0x1a17edd630>], [<networkx.classes.graph.Graph object at 0x1a17edd470>], [<networkx.classes.graph.Graph object at 0x1a17eddd30>], [<networkx.classes.graph.Graph object at 0x1a17edd3c8>], [<networkx.classes.graph.Graph object at 0x1a17edde80>], [<networkx.classes.graph.Graph object at 0x1a17edd5f8>], [<networkx.classes.graph.Graph object at 0x1a17edd550>], [<networkx.classes.graph.Graph object at 0x1a17edd828>], [<networkx.classes.graph.Graph object at 0x1a17eddfd0>], [<networkx.classes.graph.Graph object at 0x1a17eddf98>], [<networkx.classes.graph.Graph object at 0x1a17eddeb8>], [<networkx.classes.graph.Graph object at 0x1a17edd8d0>], [<networkx.classes.graph.Graph object at 0x1a17edd908>], [<networkx.classes.graph.Graph object at 0x1a17edd860>], [<networkx.classes

In [38]:
test_graph = GRAPHS['graphs']['sub-011_reward'][0]
test_graph.edges(data=True)

EdgeDataView([(0, 0, {'weight': 1.0}), (0, 1, {'weight': 0.7463512729101867}), (0, 2, {'weight': -0.8702827822271997}), (0, 3, {'weight': -0.8561540272863053}), (0, 4, {'weight': 0.10878746927332847}), (0, 5, {'weight': -0.7731923419037321}), (0, 6, {'weight': 0.27297016728853424}), (0, 7, {'weight': 0.10320204784193411}), (0, 8, {'weight': -0.8614145579233741}), (0, 9, {'weight': -0.8797196802190789}), (0, 10, {'weight': -0.5693125122430377}), (0, 11, {'weight': -0.6848794426385877}), (0, 12, {'weight': 0.6741490780290952}), (0, 13, {'weight': -0.8654721566857634}), (0, 14, {'weight': -0.7577959189021828}), (0, 15, {'weight': -0.8082411067811659}), (0, 16, {'weight': -0.8690739609157284}), (0, 17, {'weight': -0.8442732495067616}), (0, 18, {'weight': 0.792374854021122}), (0, 19, {'weight': -0.8006148099202253}), (0, 20, {'weight': -0.3686299335294704}), (0, 21, {'weight': 0.2588423211733431}), (0, 22, {'weight': -0.09730061158805152}), (0, 23, {'weight': 0.6816256513071566}), (0, 24, {

In [44]:
create_corr_network(test_graph, "spectral")

Making spectral graph


In [40]:
#Test = GRAPHS['graphs']['sub-011_reward'][0]
#Test.edges(data=True)

In [60]:
df_median

,0
0,0.0
1,0.0
2,1.0
3,1.0
4,2.0
5,2.0
6,0.0
7,0.0
8,2.0
9,2.0


In [77]:
#Convert modules to dictionary
mod_dict={}
for index, row in df_median.iterrows():
    print(index)
    mod_dict[index]= row[0]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


In [78]:
mod_dict

{0: 0.0,
 1: 0.0,
 2: 1.0,
 3: 1.0,
 4: 2.0,
 5: 2.0,
 6: 0.0,
 7: 0.0,
 8: 2.0,
 9: 2.0,
 10: 3.0,
 11: 3.0,
 12: 3.0,
 13: 3.0,
 14: 2.0,
 15: 2.0,
 16: 1.0,
 17: 1.0,
 18: 2.0,
 19: 2.0,
 20: 2.0,
 21: 2.0,
 22: 2.0,
 23: 2.0,
 24: 3.0,
 25: 3.0,
 26: 2.0,
 27: 2.0}

In [79]:
modg = community.induced_graph(mod_dict, test_graph)

In [80]:
def create_corr_network_1(G):
    #crates a list for edges and for the weights
    edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())

    #positions
    positions=nx.circular_layout(G)
    
    #Figure size
    plt.figure(figsize=(15,15))

    #draws nodes
    nx.draw_networkx_nodes(G,positions,node_color='#DA70D6',
                           node_size=500,alpha=0.8)
    
    #Styling for labels
    nx.draw_networkx_labels(G, positions, font_size=8, 
                            font_family='sans-serif')
        
    #draws the edges
    nx.draw_networkx_edges(G, positions, edge_list=edges,style='solid')
    
    # displays the graph without axis
    plt.axis('off')
    #saves image
    plt.savefig("part1.png", format="PNG")
    plt.show() 

In [81]:
create_corr_network_1(modg)

In [86]:
def create_corr_network_2(G):
   #crates a list for edges and for the weights
   edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())
   print(weights)
#     color,col_weights = zip(*nx.get_node_attributes(G,'PC').items())
   #positions
   positions=nx.circular_layout(G)
   #Figure size
   plt.figure(figsize=(15,15))
   #draws nodes
   nx.draw_networkx_nodes(G,positions,node_color='blue',
                          node_size=500,alpha=0.8, cmap= 'Set1',vmin=0 ,vmax=1 )
   #Styling for labels
   nx.draw_networkx_labels(G, positions, font_size=8,
                           font_family='sans-serif')
#     weights = weights/1000
   #draws the edges
#     print(edges)
#    weights=np.array(weights)/500
   nx.draw_networkx_edges(G, positions, edge_list=edges,style='solid', width = weights, edge_color = weights)
   # displays the graph without axis
   plt.axis('off')
   #saves image
   plt.savefig("part1.png", format="PNG")
   plt.show()

In [84]:
modg.edges(data=True)

EdgeDataView([(0.0, 0.0, {'weight': 6.191299085498064}), (0.0, 1.0, {'weight': -7.649271729084691}), (0.0, 2.0, {'weight': -4.0097392495222}), (0.0, 3.0, {'weight': -1.168303310607153}), (1.0, 1.0, {'weight': 9.80090817794927}), (1.0, 2.0, {'weight': 18.23496932816745}), (1.0, 3.0, {'weight': 5.766108115694849}), (2.0, 2.0, {'weight': 27.63525664339436}), (2.0, 3.0, {'weight': 13.201441433767778}), (3.0, 3.0, {'weight': 7.641835825856328})])

In [87]:
create_corr_network_2(modg)

(6.191299085498064, -7.649271729084691, -4.0097392495222, -1.168303310607153, 9.80090817794927, 18.23496932816745, 5.766108115694849, 27.63525664339436, 13.201441433767778, 7.641835825856328)


### Test Nodal Clustering Coefficient within module 2

In [ ]:
## Make a subgraph, visualize in cicle, the calculate clustering

In [ ]:
#this is the basic command I want to use 
print(nx.clustering(G))

# NOT USING NOW, SAVE FOR LATER 

In [ ]:
#Creates a dictionary with node names
#mapping={0:"Amygdala_L",1:"Amygdala_R", 2:"Dorsal_striatum_L", 3:"Dorsal_striatum_R", 4:"Fusiform_gyrus_L", 5:"Fusiform_gyrus_R", 6:"Hippocampus_L", 7:"Hippocampus_R", 8:"Insula_L",
#         9:"Insula_R", 10:"Intracalcarine_cortex_L", 11:"Intracalcarine_cortex_R", 12:"lOFC_R", 13: "mOFC_L", 14:"mOFC_R", 15:"Oral_somatosensory_cortex_L", 16:"Oral_somatosensory_cortex_R", 17:"Precuneus_L", 
#         18:"Precuneus_R", 19:"Ventral_striatum_L", 20:"Ventral_striatum_R", 21:"vlPFC_L", 22:"vlPFC_R", 23:"vlThalamus_L" , 24:"vlThalamus_R", 25: "vmPFC_L", 26: "vmPFC_R"}

#relabels the nodes to match the ROI names
#G = nx.relabel_nodes(GRAPHS, mapping)

### Combine Participant Graphs into one "mega" graph

In [ ]:
def make_total_graphs(dict_o_data):
    mylist=[]
    for key, val_list in dict_o_data.items():
#         print(key)
        for i in val_list:
            j = i[0]
            cor_matrix = np.asarray(j)
            mylist.append(cor_matrix)
    x = np.stack(mylist)
    #mu=np.mean(x)
    #return(mu)

In [ ]:
#male a copy of graphs
dict2 = GRAPHS.copy()

#get rid of the columns we don't need
del dict2["mean_FC"]
del dict2["edges"]
del dict2["correlations"]

In [ ]:
#check to see 
dict2.keys()

In [ ]:
dict2['graphs']

^ This isn't working for me because the list is nested in the dictorary values

In [ ]:
big_graph = make_total_graphs(dict2)

^ "big_graph" is empty

Trying to make a list of graph objects instead: 

In [ ]:
graph_list=[]

for key,value in GRAPHS['graphs'].items():
    graph_list.append(value[0])

In [ ]:
print(graph_list)

In [ ]:
#This function is meant for one graph. 
def create_corr_network(G, corr_direction, min_correlation):
    ##Creates a copy of the graph
    H = G.copy()
    
    ##Checks all the edges and removes some based on corr_direction
    for stock1, stock2, weight in list(G.edges(data=True)):
        ##if we only want to see the positive correlations we then delete the edges with weight smaller than 0        
        if corr_direction == "positive":
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] <0 or weight["weight"] < min_correlation:
                H.remove_edge(stock1, stock2)
        ##this part runs if the corr_direction is negative and removes edges with weights equal or largen than 0
        else:
            ####it adds a minimum value for correlation. 
            ####If correlation weaker than the min, then it deletes the edge
            if weight["weight"] >=0 or weight["weight"] > min_correlation:
                H.remove_edge(stock1, stock2)
                
    
    #creates a list for edges and for the weights
    edges,weights = zip(*nx.get_edge_attributes(H,'weight').items())
    
    ### increases the value of weights, so that they are more visible in the graph
    weights = tuple([(1+abs(x))**1 for x in weights])
    
    #####calculates the degree of each node
    d = nx.degree(H)
    #####creates list of nodes and a list their degrees that will be used later for their sizes
    nodelist, node_sizes = zip(*list(d))
    return(H)

In [ ]:
test = create_corr_network(graph_list[0], "positive",0)